In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import datetime
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['font.serif'] = ['SimHei']
import seaborn as sns
%matplotlib inline
p = sns.color_palette()
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})

#https://github.com/mwaskom/seaborn/issues/1009

import os
print (os.path.abspath(os.curdir))

/home/fg/IdeaProjects/jd/Untitled Folder


In [2]:
t_loan="../input/t_loan.csv"

In [3]:
loan_info = pd.read_csv(t_loan,low_memory=False)
loan_info.info()
loan_info.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202902 entries, 0 to 202901
Data columns (total 4 columns):
uid            202902 non-null int64
loan_time      202902 non-null object
loan_amount    202902 non-null float64
plannum        202902 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 6.2+ MB


,uid,loan_time,loan_amount,plannum
0,12135,2016-08-03 00:05:26,3.862595,1
1,41403,2016-08-03 00:13:25,5.584137,3
2,74458,2016-08-03 00:13:58,4.723017,1


In [4]:
# user_loan_info=pd.merge(loan_info,user_info.loc[:,['uid','limit']],how='left', on = "uid")
# user_loan_info.info()
# user_loan_info=user_loan_info.query('loan_amount  <= limit')
# loan_info=user_loan_info.loc[:,['uid', 'loan_time', 'loan_amount', 'plannum']]
# loan_info.info()
loan_info['payback']=loan_info.loan_amount/loan_info.plannum

In [5]:
loan_info.head(5)

,uid,loan_time,loan_amount,plannum,payback
0,12135,2016-08-03 00:05:26,3.862595,1,3.862595
1,41403,2016-08-03 00:13:25,5.584137,3,1.861379
2,74458,2016-08-03 00:13:58,4.723017,1,4.723017
3,12959,2016-08-03 00:19:33,3.862595,1,3.862595
4,89641,2016-08-03 00:23:13,4.292651,1,4.292651


In [6]:
loan_info['month'] = pd.DatetimeIndex(loan_info.loan_time).month
# loan_info['day'] = pd.DatetimeIndex(df_train.datetime).dayofweek

In [7]:
loan_info_08M=loan_info[loan_info['month']==8]
loan_info_09M=loan_info[loan_info['month']==9]
loan_info_10M=loan_info[loan_info['month']==10]
loan_info_11M=loan_info[loan_info['month']==11]

In [8]:
def feature_sum_count_max_min_mean_median_std_var( dataset, month):
    d=dataset
    gb= d.loc[:,['uid','payback' ]].groupby('uid',as_index=False)

    loan_sum=gb.sum()
    loan_sum.columns=['uid', str(month)+'M_payback_sum']
    feature=loan_sum

    loan_max=gb.max()
    loan_max.columns=['uid', str(month)+'M_payback_max']
    feature=pd.merge(feature, loan_max,how='left', on = 'uid')

    loan_min=gb.min()
    loan_min.columns=['uid', str(month)+'M_payback_min']
    feature=pd.merge(feature, loan_min,how='left', on = 'uid')

    loan_mean=gb.mean()
    loan_mean.columns=['uid', str(month)+'M_payback_mean']
    feature=pd.merge(feature, loan_mean,how='left', on = 'uid')

    return feature

In [9]:
loan_info_summary=loan_info.loc[:,['uid']].drop_duplicates()
feature_08M_summary=feature_sum_count_max_min_mean_median_std_var(loan_info_08M, 8)
loan_info_summary=pd.merge(loan_info_summary, feature_08M_summary,how='left', on = 'uid')
feature_09M_summary=feature_sum_count_max_min_mean_median_std_var(loan_info_09M, 9)
loan_info_summary=pd.merge(loan_info_summary, feature_09M_summary,how='left', on = 'uid')
feature_10M_summary=feature_sum_count_max_min_mean_median_std_var(loan_info_10M, 10)
loan_info_summary=pd.merge(loan_info_summary, feature_10M_summary,how='left', on = 'uid')
feature_11M_summary=feature_sum_count_max_min_mean_median_std_var(loan_info_11M, 11)
loan_info_summary=pd.merge(loan_info_summary, feature_11M_summary,how='left', on = 'uid')
loan_info_summary.fillna(0,inplace=True)
loan_info_summary.fillna(0.0,inplace=True)
# train_data.fillna({'loan_sum':0.0},inplace=True)

In [10]:
# loan_info_summary['9M_8M_loan_amount']=loan_info_summary['9M_loan_amount_sum']-loan_info_summary['8M_loan_amount_sum']
# loan_info_summary['10M_9M_loan_amount']=loan_info_summary['10M_loan_amount_sum']-loan_info_summary['9M_loan_amount_sum']
# loan_info_summary['11M_10M_loan_amount']=loan_info_summary['11M_loan_amount_sum']-loan_info_summary['10M_loan_amount_sum']
loan_info_summary.head(5)

,uid,8M_payback_sum,8M_payback_max,8M_payback_min,8M_payback_mean,9M_payback_sum,9M_payback_max,9M_payback_min,9M_payback_mean,10M_payback_sum,10M_payback_max,10M_payback_min,10M_payback_mean,11M_payback_sum,11M_payback_max,11M_payback_min,11M_payback_mean
0,12135,5.293478,3.862595,1.430884,2.646739,5.335297,1.801805,0.414570,1.067059,6.050874,3.862595,0.715442,1.512718,5.721805,4.544373,0.357721,1.430451
1,41403,59.131246,5.885770,1.574339,4.548557,55.375187,6.066890,4.292651,5.034108,21.479072,5.741132,4.723017,5.369768,18.824048,5.931813,1.764051,4.706012
2,74458,30.049485,4.723017,3.862595,4.292784,33.051037,4.292651,3.862595,4.131380,35.024173,5.153538,3.862595,4.378022,69.544383,4.723017,3.862595,4.346524
3,12959,25.034687,4.861602,3.862595,4.172448,19.217180,4.974843,4.292651,4.804295,19.217180,4.974843,4.292651,4.804295,24.192022,4.974843,4.292651,4.838404
4,89641,159.331521,4.544373,4.292651,4.306257,137.364822,4.292651,4.292651,4.292651,55.815470,4.723017,4.292651,4.651289,35.476437,5.292154,4.292651,5.068062


In [11]:
loan_info_summary.fillna(0.0,inplace=True)
loan_info_summary.fillna(0,inplace=True)
loan_info_summary.to_csv("../feature/mid/loan_info_summary_payback.csv",index=None,encoding='utf-8')

In [9]:
loan_info_summary.apply(lambda x: sum(x.isnull()))

uid                      0
8M_loan_amount_sum       0
8M_plannum_sum           0
8M_loan_amount_count     0
8M_plannum_count         0
8M_loan_amount_max       0
8M_plannum_max           0
8M_loan_amount_min       0
8M_plannum_min           0
8M_loan_amount_mean      0
8M_plannum_mean          0
9M_loan_amount_sum       0
9M_plannum_sum           0
9M_loan_amount_count     0
9M_plannum_count         0
9M_loan_amount_max       0
9M_plannum_max           0
9M_loan_amount_min       0
9M_plannum_min           0
9M_loan_amount_mean      0
9M_plannum_mean          0
10M_loan_amount_sum      0
10M_plannum_sum          0
10M_loan_amount_count    0
10M_plannum_count        0
10M_loan_amount_max      0
10M_plannum_max          0
10M_loan_amount_min      0
10M_plannum_min          0
10M_loan_amount_mean     0
10M_plannum_mean         0
11M_loan_amount_sum      0
11M_plannum_sum          0
11M_loan_amount_count    0
11M_plannum_count        0
11M_loan_amount_max      0
11M_plannum_max          0
1

In [13]:
loan_info.select_dtypes(include=['int64']).describe().T.\
assign(missing_pct=loan_info.apply(lambda x : (len(x)-x.count())/float(len(x))))

ValueError: Cannot describe a DataFrame without columns

In [10]:
loan_info.head(3)

,uid,loan_time,loan_amount,plannum,month
0,12135,2016-08-03 00:05:26,3.862595,1,8
1,41403,2016-08-03 00:13:25,5.584137,3,8
2,74458,2016-08-03 00:13:58,4.723017,1,8


In [7]:
user_loan_info.head(3)

,uid,loan_time,loan_amount,plannum,limit
0,12135,2016-08-03 00:05:26,3.862595,1,6.292055
1,41403,2016-08-03 00:13:25,5.584137,3,6.292055
2,74458,2016-08-03 00:13:58,4.723017,1,5.974677
